# CoxPH tests

Test one or more classifier models against held-out dataset Using held-out test features and evaluates the peformance of the estimated model.<br>
Can be part of a kubeflow pipeline as a test step that is run post EDA and training/validation cycles

Information on how to create the model and dataset exists [HERE](https://github.com/mlrun/functions/blob/development/coxph_trainer/coxph_trainer.ipynb)

## lets download the test dataset and the pretrained model

In [1]:
import mlrun
import os

In [2]:
# download from s3 or something 
cox_model_loc = 'https://s3.wasabisys.com/iguazio/models/coxph/cox-model.pkl'
cox_data_loc = 'https://s3.wasabisys.com/iguazio/data/coxph/tenured-test-set.csv'

if(not(os.path.exists(os.getcwd()+'/artifacts'))):
    saved_models_directory = os.path.join(os.getcwd(), 'artifacts')
    os.makedirs(saved_models_directory, exist_ok=1)
    
    ! wget -nc -P {saved_models_directory} {cox_model_loc}
    ! wget -nc -P {saved_models_directory} {cox_data_loc}
else:
    print(f"files already exists in path {os.getcwd()+'/artifacts'}")

--2021-08-15 11:10:49--  https://s3.wasabisys.com/iguazio/models/coxph/cox-model.pkl
Resolving s3.wasabisys.com (s3.wasabisys.com)... 38.27.106.51, 38.27.106.53
Connecting to s3.wasabisys.com (s3.wasabisys.com)|38.27.106.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50481 (49K) [application/octet-stream]
Saving to: ‘/User/test/functions/coxph_test/artifacts/cox-model.pkl’

cox-model.pkl       100%[===================>]  49.30K  --.-KB/s    in 0.09s   

2021-08-15 11:10:50 (537 KB/s) - ‘/User/test/functions/coxph_test/artifacts/cox-model.pkl’ saved [50481/50481]

--2021-08-15 11:10:51--  https://s3.wasabisys.com/iguazio/data/coxph/tenured-test-set.csv
Resolving s3.wasabisys.com (s3.wasabisys.com)... 38.27.106.51, 38.27.106.53
Connecting to s3.wasabisys.com (s3.wasabisys.com)|38.27.106.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14858 (15K) [application/vnd.ms-excel]
Saving to: ‘/User/test/functions/coxph_test/artifacts/tenu

## importing the function

In [3]:
fn = mlrun.import_function("hub://coxph_test").apply(mlrun.auto_mount())

## running the function

In [4]:
fn.run(params = {"label_column": "labels",
                 "plots_dest": "/plots"},
       inputs = {"test_set": 'artifacts/tenured-test-set.csv',
                 "models_path": os.getcwd()+"/artifacts"},
       workdir = os.getcwd(),
       artifact_path = os.getcwd() + '/artifacts')

> 2021-08-15 11:10:52,104 [info] starting run coxph-test-cox_test uid=d84016166928402fa4b4265f2c799be7 DB=http://mlrun-api:8080
> 2021-08-15 11:10:52,215 [info] Job is running in the background, pod: coxph-test-cox-test-zq6s8
> 2021-08-15 11:10:58,672 [info] cox tester not implemented
> 2021-08-15 11:10:58,696 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...2c799be7,0,Aug 15 11:10:58,completed,coxph-test-cox_test,v3io_user=danikind=jobowner=danihost=coxph-test-cox-test-zq6s8,test_setmodels_path,label_column=labelsplots_dest=/plots,,cox-test-summary


to track results use .show() or .logs() or in CLI: 
!mlrun get run d84016166928402fa4b4265f2c799be7 --project default , !mlrun logs d84016166928402fa4b4265f2c799be7 --project default
> 2021-08-15 11:11:01,402 [info] run executed, status=completed


## test summary

In [5]:
import pandas as pd
df = pd.read_csv("artifacts/cox-test-summary.csv")
df

,covariate,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
0,gender,0.712986,2.040073e+00,0.343471,0.039795,1.386176,1.040598,3.999528,2.075826,0.037910,4.721274
1,senior,-0.330137,7.188252e-01,0.444705,-1.201743,0.541468,0.300670,1.718528,-0.742374,0.457861,1.127018
2,partner,-0.394449,6.740516e-01,0.432243,-1.241630,0.452732,0.288913,1.572603,-0.912562,0.361473,1.468041
3,deps,0.616373,1.852199e+00,0.499075,-0.361797,1.594543,0.696424,4.926080,1.235031,0.216819,2.205436
4,MultipleLines,-0.787885,4.548059e-01,1.087536,-2.919417,1.343648,0.053965,3.832999,-0.724467,0.468779,1.093020
5,OnlineSecurity,-0.766683,4.645512e-01,1.299746,-3.314139,1.780772,0.036365,5.934435,-0.589872,0.555277,0.848721
6,OnlineBackup,-0.466691,6.270740e-01,0.949068,-2.326829,1.393448,0.097605,4.028715,-0.491736,0.622906,0.682914
7,DeviceProtection,-0.412620,6.619136e-01,1.083731,-2.536694,1.711453,0.079128,5.537002,-0.380741,0.703396,0.507591
8,TechSupport,0.509756,1.664885e+00,1.168080,-1.779638,2.799150,0.168699,16.430675,0.436405,0.662543,0.593915
9,PaperlessBilling,0.349970,1.419025e+00,0.408827,-0.451317,1.151257,0.636789,3.162165,0.856033,0.391980,1.351150
